![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/DEID_PHI_TEXT_MULTI.ipynb)

## **Deidentification Model for Different Languages**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## **Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [7]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import PretrainedPipeline


from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


## **Sample Examples for Different languages**

In [4]:
text_english = ["""Record date: 2069-04-07
Mr. Villegas is seen today.  I have not seen him since November. 
About three weeks ago he stopped his Prednisone on his own because
he was gaining weight.  He does feel that his shoulders are
definitely improved.
On examination today, BP 120/80.  His joint examination is much
improved with better ROM of the shoulders and no peripheral joint
synovitis.  
Clinical Impression:
1:  Inflammatory arthritis - possibly RA - with response noted to
Hydroxychloroquine along with Prednisone.  He has stopped the
Prednisone, and I would not restart it yet.  
2:  New onset of symptoms suspicious for right-sided carotid
disease.  Will arrange for carotid ultrasound studies.  Patient
advised to call me if he develops any worsening symptoms.
Xzavian G. Tavares, M.D.
XGT:holmes
DD: 04/07/69
DT: 04/15/69
DV: 04/07/69"""]



text_german = ["""Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus eingeliefert. 
Herr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.Persönliche Daten :ID-Nummer: T0110053F Platte A-BC124 Kontonummer: DE89370400440532013000 SSN : 13110587M565 Lizenznummer: B072RRE2I55 Adresse : St.Johann-Straße 13 19300
"""]


text_spanish = ["""Nombre: Danilo.
Apellidos: Ramos Portillo.
NHC: 4376372.
NASS: 35 61051794 56.
Domicilio: Calle Fernando Higueras, 27, 7B.
Localidad/ Provincia: Madrid.
CP: Madrid 2805528055.
Datos asistenciales.
Fecha de nacimiento: 15/07/1979.
País: España.
Edad: 38 años Sexo: H.
Fecha de Ingreso: 04/03/2014.
Episodio: 82034567902.
Médico: Fabio Jiménez Borrero NºCol: 28 28 57986.
Informe clínico del paciente.
Historia Actual: Paciente varón de 38 años, valorado en consulta por coxalgia derecha de año y medio de evolución, de características mecánicas. Refiere dolor con práctica de ejercicio físico y leve limitación de la movilidad en ciertas posturas. Mecánico de profesión, en su tiempo libre realizaba bicicleta (40 km diarios en fines de semana), carrera 3 veces por semana durante 50 minutos y marcha de una hora los días que no realizaba carrera. El dolor se ha ido incrementando progresivamente, realizando, en la actualidad, solamente marcha.
Exploración física: A la exploración física destaca un dolor a la palpación en tercio medio de la región inguinal derecha con dolor a la flexión, aducción y rotaciones sobre todo interna (maniobra de choque positiva). El balance articular de caderas es de flexión 110o bilateral, rotación interna de 10o derecha, 40o izquierda y rotación externa de 30o derecha, 40o izquierda. El balance muscular en la escala de Daniels es de 5/5 global bilateral. No presenta dolor a la palpación de puntos ciáticos y las maniobras de Valleix, Fabere, Lasegue y Bragard son negativas. Tampoco presenta dolor a la palpación en la cintilla iliotibial y el test de Ober es negativo. En la exploración de la marcha observamos ausencia de claudicación y el apoyo monopodal alterno derecho-izquierdo es estable, con Trendelenburg negativo. Presenta dolor para la marcha con flexión forzada de cadera y rodilla.
Pruebas complementarias: La radiología simple anteroposterior (AP) presentaba una deformidad en forma de joroba en región cérvico-cefálica femoral derecha. La analítica solicitada con hemograma, bioquímica, hormonas tiroideas, anticuerpos y reactantes de fase aguda era normal.
Ante estos hallazgos se solicita resonancia magnética de cadera en la que se visualiza leve alteración subcondral, con edema óseo en cadera derecha, con irregularidad cortical y leve sinovitis en interlínea articular, con distensión de la bursa ilio-psoas. La cabeza femoral presenta un pequeño islote óseo y el ángulo alfa es mayor de 50o. La artrografía muestra un pinzamiento de la interlinea articular antero-superior y posteroinferior, un incipiente collar osteofitario femoral con pequeñas lesiones subcondrales antero-superiores de cabeza femoral y pequeños focos de edema en acetábulo anterosuperior. No se visualiza necrosis avascular ni lesiones en el labrum. También se visualiza una lesión quística yuxtaarticular en relación con trocánter mayor.
Evolución: Con el diagnóstico de síndrome de pinzamiento femoroacetabular tipo LEVA (también conocido por CAM) se le prescribe al paciente tratamiento con magnetoterapia (15 sesiones), para mejorar la clínica del dolor y el trofismo óseo.
Se remite a unidad de artroscopia del servicio de traumatología para valoración de cirugía artroscópica.
Diagnóstico Principal: síndrome de pinzamiento femoroacetabular tipo LEVA
Tratamiento: Se pauta tratamiento farmacológico con asociación de sulfato de glucosamina-condroitin sulfato oral 400 mg cada 12 horas durante 3 meses y antiinflamatorios no esteroideos a demanda si presenta dolor. Se le indica reposo deportivo para evitar choque en la articulación, con actividad física activa controlada, recomendando no sobrepasar los 90 grados de flexión de la cadera y evitar las rotaciones. Acudir a cita para revisión una vez terminados los sulfatos de glucosamina-condroitin para ver la mejoría clínica, deberá llamar al numero de atención y citas previas de hospital 914 90 66 60
Remitido por: Dr. Fabio Jiménez Borrero. Hospital Universitario La Paz. Paseo de la Castellana, 261, 28046 Madrid. Teléfono: 917 27 70 00. Correo electrónico: fabio_jimenezb@salud.madrid.org 
"""]



text_italian = ["""PZ: Giancarlo Binaghi
CODICE FISCALE: MVANSK92F09W408A
INDIRIZZO: Viale Burcardo 7
CITTÀ : Napoli
CODICE POSTALE: 80139
DATA DI NASCITA: 03/03/1946
ETÀ: 70 anni 
SESSO: M
EMAIL: gcbinaghi@tim.it
DATA DI AMMISSIONE: 12/12/2016
DOTTORE: Eva Viviani
RAPPORTO CLINICO: 70 anni, pensionato, senza allergie farmacologiche note, che presenta la seguente storia: ex incidente sul lavoro con fratture vertebrali e costali; operato per la malattia di Dupuytren alla mano destra e un bypass ileo-femorale sinistro; diabete di tipo II, ipercolesterolemia e iperuricemia; alcolismo attivo, fuma 20 sigarette/giorno.
È stato indirizzato a noi perché ha presentato un'ematuria macroscopica post-evacuazione in un'occasione e una microematuria persistente in seguito, con un'evacuazione normale.
L'esame fisico ha mostrato buone condizioni generali, con addome e genitali normali; l'esame digitale rettale era coerente con un adenoma prostatico di grado I/IV.
L'analisi delle urine ha mostrato 4 globuli rossi/campo e 0-5 leucociti/campo; il resto del sedimento era normale.
L'emocromo è normale; la biochimica ha mostrato una glicemia di 169 mg/dl e trigliceridi 456 mg/dl; la funzione epatica e renale sono normali. PSA di 1,16 ng/ml.
INDIRIZZATO A: Dott. Bruno Ferrabosco - ASL Napoli 1 Centro, Dipartimento di Endocrinologia e Nutrizione - Stretto Scamarcio 320, 80138 Napoli
EMAIL: bferrabosco@poste.it
"""]


text_french = ["""PRENOM : Jean
NOM : Dubois
NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058
ADRESSE : 18 Avenue Matabiau
VILLE : Grenoble
CODE POSTAL : 38000
DATE DE NAISSANCE : 03/03/1946
Âge : 70 ans 
Sexe : H
COURRIEL : jdubois@hotmail.fr
DATE D'ADMISSION : 12/12/2016
MÉDÉCIN : Dr Michel Renaud
RAPPORT CLINIQUE : 70 ans, retraité, sans allergie médicamenteuse connue, qui présente comme antécédents : ancien accident du travail avec fractures vertébrales et des côtes ; opéré de la maladie de Dupuytren à la main droite et d'un pontage ilio-fémoral gauche ; diabète de type II, hypercholestérolémie et hyperuricémie ; alcoolisme actif, fume 20 cigarettes / jour.
Il nous a été adressé car il présentait une hématurie macroscopique postmictionnelle à une occasion et une microhématurie persistante par la suite, avec une miction normale.
L'examen physique a montré un bon état général, avec un abdomen et des organes génitaux normaux ; le toucher rectal était compatible avec un adénome de la prostate de grade I/IV.
L'analyse d'urine a montré 4 globules rouges/champ et 0-5 leucocytes/champ ; le reste du sédiment était normal.
Hémogramme normal ; la biochimie a montré une glycémie de 169 mg/dl et des triglycérides de 456 mg/dl ; les fonctions hépatiques et rénales étaient normales. PSA de 1,16 ng/ml.
ADDRESSÉ À : Dre Marie Breton - Centre Hospitalier de Bellevue Service D'Endocrinologie et de Nutrition - Rue Paulin Bussières, 38000 Grenoble
COURRIEL : mariebreton@chb.fr
"""]


text_portuguese = ["""Dados do paciente.
Nome: Mauro.
Apelido: Gonçalves.
NISS: 26 63514095.
Endereço: Calle Miguel Benitez 90.
CÓDIGO POSTAL: 28016.
Dados de cuidados.
Data de nascimento: 03/03/1946.
País: Portugal.
Idade: 70 anos Sexo: M.
Data de admissão: 12/12/2016.
Doutor: Ignacio Navarro Cuéllar NºCol: 28 28 70973.
Relatório clínico do paciente: Paciente de 70 anos, mineiro reformado, sem alergias medicamentosas conhecidas, que apresenta como história pessoal: acidente de trabalho antigo com fracturas vertebrais e das costelas; operado por doença de Dupuytren na mão direita e iliofemoral esquerda; Diabetes Mellitus tipo II, hipercolesterolemia e hiperuricemia; alcoolismo activo, fumador de 20 cigarros / dia.
Foi encaminhado dos cuidados primários porque apresentou uma vez hematúria macroscópica pós-morte e depois microhaematúria persistente, com micturição normal.
O exame físico mostrou um bom estado geral, com abdómen e genitália normais; o exame rectal foi compatível com adenoma de próstata de grau I/IV.
A urinálise mostrou 4 glóbulos vermelhos/campo e 0-5 leucócitos/campo; o resto do sedimento estava normal.
Hemograma normal; a bioquímica mostrou glicemia de 169 mg/dl e triglicéridos de 456 mg/dl; função hepática e renal normal. PSA de 1,16 ng/ml.
A citologia da urina era repetidamente desconfiada por malignidade.
A radiografia simples abdominal mostra alterações degenerativas na coluna lombar e calcificações vasculares tanto no hipocôndrio como na pélvis.
A ecografia urológica revelou cistos corticais simples no rim direito, uma bexiga inalterada com boa capacidade e uma próstata com 30g de peso.
O IVUS mostrou normofuncionalismo renal bilateral, calcificações na silhueta renal direita e ureteres artrosados com imagens de adição no terço superior de ambos os ureteres, relacionadas com pseudodiverticulose ureteral. O cistograma mostra uma bexiga com boa capacidade, mas com paredes trabeculadas em relação à bexiga de stress. A tomografia computorizada abdominal é normal.
A cistoscopia revelou a existência de pequenos tumores na bexiga, e a ressecção transuretral foi realizada com o resultado anatomopatológico do carcinoma urotelial superficial da bexiga.
Referido por: Miguel Santos - Avenida dos Aliados, 22 Portugal E-mail: nnavcu@hotmail.com.
"""]


text_romanian = ["""TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP 
Medic Laborator : BURIAN MONICA 
PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : 11 FEBRUARIE 2022 
Nume si prenume : RUS OLIMPIU DUMITRU , 49 ani, Sex : M
TEL : +40 67 5745, 
E-mail : olimpiu_d@gmail.com
CNP : 1730104060763
Licență : E000198985A, Înmatriculare : CD32156, Cont : LZWZ7170951927104999,
Solicitare : Angio CT coronarian 
kg : 100 Dg .
de trimitere Stare post-AVCI .
Stare post-trombectomie .
Proces inlocuitor de spatiu ventricular stang
Trimis de : Sectia Clinica Neurologie 1a SCJU Tirgu Mures
Medic : Dr.Melanie Dana
CONCLUZII : - Tromb ventricular stang de 20/40/17 mm , - Leziuni sechelar ischemice in segmentele apicale septal si inferior , cu aspect anevrismal apical VS - Leziune non-ischemica/MINOCA ?
48	mediocardiaca septala - Examenul AngioCT coronarian nu pune in evidenta ateromatoza sau stenoze coronariene semnificative.
"""]

# **🔎Define Spark NLP pipeline**

In [8]:
lang_dict = {"en" : text_english,
             "de" : text_german,
             "es" : text_spanish,
             "it" : text_italian,
             "fr" : text_french,
             "pt" : text_portuguese,
             "ro" : text_romanian
             }

In [9]:
import pandas as pd

for  lang, text in lang_dict.items():
    if lang == "es":
        deid_pipeline = PretrainedPipeline("clinical_deidentification_augmented", "es", "clinical/models")
        deid_res = deid_pipeline.annotate(text)[0]
        df= pd.DataFrame(list(zip(deid_res["sentence"], deid_res["masked"], deid_res["masked_with_chars"], deid_res["masked_fixed_length_chars"], deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])
        
    else:
        deid_pipeline = PretrainedPipeline("clinical_deidentification", lang, "clinical/models")
        deid_res = deid_pipeline.annotate(text)[0]
        df= pd.DataFrame(list(zip(deid_res["sentence"], deid_res["masked"],
                          deid_res["masked_with_chars"], deid_res["masked_fixed_length_chars"], deid_res["obfuscated"])),
                 columns= ["Sentence", "Masked", "Masked with Chars", "Masked with Fixed Chars", "Obfuscated"])
        
    
    lang = lang.upper()
    print("\n", f"DE-IDENTIFICATION --->  {lang} : ")
    
    display(df.head()) 

clinical_deidentification download started this may take some time.
Approx size to download 1.6 GB
[OK!]

 DE-IDENTIFICATION --->  EN : 


,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,Record date: 2069-04-07,Record date: <DATE>,Record date: [********],Record date: ****,Record date: 2069-04-21
1,Mr. Villegas is seen today.,Mr. <PATIENT> is seen today.,Mr. [******] is seen today.,Mr. **** is seen today.,Mr. Selena Rom is seen today.
2,I have not seen him since November.,I have not seen him since <DATE>.,I have not seen him since [******].,I have not seen him since ****.,I have not seen him since 12-27-1997.
3,About three weeks ago he stopped his Prednisone on his own because\nhe was gaining weight.,About three weeks ago he stopped his Prednisone on his own because\nhe was gaining weight.,About three weeks ago he stopped his Prednisone on his own because\nhe was gaining weight.,About three weeks ago he stopped his Prednisone on his own because\nhe was gaining weight.,About three weeks ago he stopped his Prednisone on his own because\nhe was gaining weight.
4,He does feel that his shoulders are\ndefinitely improved.,He does feel that his shoulders are\ndefinitely improved.,He does feel that his shoulders are\ndefinitely improved.,He does feel that his shoulders are\ndefinitely improved.,He does feel that his shoulders are\ndefinitely improved.


clinical_deidentification download started this may take some time.
Approx size to download 1.2 GB
[OK!]

 DE-IDENTIFICATION --->  DE : 


,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,Zusammenfassung : Michael Berger wird am Morgen des 12 Dezember 2018 ins St.Elisabeth Krankenhaus eingeliefert.,Zusammenfassung : <PATIENT> wird am Morgen des <DATE> ins <HOSPITAL> eingeliefert.,Zusammenfassung : [************] wird am Morgen des [**************] ins [**********************] eingeliefert.,Zusammenfassung : **** wird am Morgen des **** ins **** eingeliefert.,Zusammenfassung : Hollmann Burmeister wird am Morgen des 11-01-1970 ins FRANKLIN WOODS COMMUNITY HOSPITAL eingeliefert.
1,Herr Michael Berger ist 76 Jahre alt und hat zu viel Wasser in den Beinen.,Herr <PATIENT> ist <AGE> Jahre alt und hat zu viel Wasser in den Beinen.,Herr [************] ist ** Jahre alt und hat zu viel Wasser in den Beinen.,Herr **** ist **** Jahre alt und hat zu viel Wasser in den Beinen.,Herr Hollmann Burmeister ist 57 Jahre alt und hat zu viel Wasser in den Beinen.
2,Persönliche Daten :ID-Nummer: T0110053F Platte A-BC124 Kontonummer: DE89370400440532013000 SSN : 13110587M565 Lizenznummer: B072RRE2I55 Adresse : St.Johann-Straße 13 19300,Persönliche Daten :ID-Nummer: <ID> Platte <PLATE> Kontonummer: <ACCOUNT> SSN : <SSN> Lizenznummer: <DLN> Adresse : <STREET> <ZIP>,Persönliche Daten :ID-Nummer: [*******] Platte [*****] Kontonummer: [********************] SSN : [**********] Lizenznummer: [*********] Adresse : [*****************] [***],Persönliche Daten :ID-Nummer: **** Platte **** Kontonummer: **** SSN : **** Lizenznummer: **** Adresse : **** ****,Persönliche Daten :ID-Nummer: L6043236 Platte QA348G Kontonummer: 192837465738 SSN : 999-30-4262 Lizenznummer: S99913378 Adresse : Guntram-Hofmann-Gasse 6 50488


clinical_deidentification_augmented download started this may take some time.
Approx size to download 268.2 MB
[OK!]

 DE-IDENTIFICATION --->  ES : 


...

,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,PRENOM : Jean,PRENOM : <PATIENT>,PRENOM : [**],PRENOM : ****,PRENOM : Mme Ollivier
1,NOM : Dubois,NOM : <PATIENT>,NOM : [****],NOM : ****,NOM : Mme Traore
2,NUMÉRO DE SÉCURITÉ SOCIALE : 1780160471058,NUMÉRO DE SÉCURITÉ SOCIALE : <SSN>,NUMÉRO DE SÉCURITÉ SOCIALE : [***********],NUMÉRO DE SÉCURITÉ SOCIALE : ****,NUMÉRO DE SÉCURITÉ SOCIALE : 164033818514436
3,ADRESSE : 18 Avenue Matabiau,ADRESSE : <STREET>,ADRESSE : [****************],ADRESSE : ****,"ADRESSE : 731, boulevard de Legrand"
4,VILLE : Grenoble,VILLE : <CITY>,VILLE : [******],VILLE : ****,VILLE : Sainte Antoine


clinical_deidentification download started this may take some time.
Approx size to download 1.2 GB
[OK!]

 DE-IDENTIFICATION --->  PT : 


,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,Dados do paciente.,Dados do <SEX>.,Dados do [******].,Dados do ****.,Dados do Homen.
1,Nome: Mauro.,Nome: <PATIENT>.,Nome: [***].,Nome: ****.,Nome: Tiago Santos.
2,Apelido: Gonçalves.,Apelido: <PATIENT>.,Apelido: [*******].,Apelido: ****.,Apelido: Marcos Alves.
3,NISS: 26 63514095.,NISS: <ID>.,NISS: **********].,NISS: ****.,NI00351677174789: 566-445.
4,Endereço: Calle Miguel Benitez 90.\nCÓDIGO POSTAL: 28016.,Endereço: <STREET>.\nCÓDIGO POSTAL: <ZIP>.,Endereço: [*********************].\nCÓDIGO POSTAL: [***].,Endereço: ****.\nCÓDIGO POSTAL: ****.,"Endereço: Avenida da Liberdade, 17.\nCÓDIGO POSTAL: 98584."


clinical_deidentification download started this may take some time.
Approx size to download 1.1 GB
[OK!]

 DE-IDENTIFICATION --->  RO : 


,Sentence,Masked,Masked with Chars,Masked with Fixed Chars,Obfuscated
0,TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP,TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP,TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP,TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP,TOMOGRAFIA COMPUTERIZATA A CREIERULUI INVESTIGATII REZULTAT DLP
1,Medic Laborator : BURIAN MONICA,Medic Laborator : <DOCTOR>,Medic Laborator : [***********],Medic Laborator : ****,Medic Laborator : A.N. Diaconescu
2,PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : 11 FEBRUARIE 2022,PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : <DATE>,PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : [***************],PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : ****,PROTOCOL DE EXPLORARE PRIN COMPUTER TOMOGRAFIE DATA : 11-17-1970
3,"Nume si prenume : RUS OLIMPIU DUMITRU , 49 ani, Sex : M\nTEL : +40 67 5745, \nE-mail : olimpiu_d@gmail.com","Nume si prenume : <PATIENT> , <AGE> ani, Sex : M\nTEL : <PHONE>, \nE-mail : <EMAIL>","Nume si prenume : [*****************] , ** ani, Sex : M\nTEL : [*********], \nE-mail : [*****************]","Nume si prenume : **** , **** ani, Sex : M\nTEL : ****, \nE-mail : ****","Nume si prenume : Dragolici Adina Nutoaica , 83 ani, Sex : M\nTEL : 0770 664 874, \nE-mail : lstancu@zzup.ro"
4,CNP : 1730104060763,CNP : <IDNUM>,CNP : [***********],CNP : ****,CNP : 263171565812
